In [1]:
import concurrent.futures as cf
import itertools

import gps2var
import numpy as np
import pyproj
import rasterio

In [2]:
DATA_PATH = "wildareas-v3-2009-human-footprint.tif"

In [3]:
np.random.seed(42)
lat, lon = np.random.uniform(-1, 1, size=(10000, 2)).T

## Basic synchronous reading

In [4]:
def time_rasterio_sample(lon, lat):
    """rasterio.DatasetReader.sample()"""
    with rasterio.open(DATA_PATH) as dataset:
        transformer = pyproj.Transformer.from_crs("EPSG:4326", dataset.crs, always_xy=True)
        %timeit list(dataset.sample(zip(*transformer.transform(lon, lat))))

In [5]:
def time_reader(lon, lat):
    """RasterValueReader()"""
    with gps2var.RasterValueReader(DATA_PATH) as reader:
        %timeit reader.get(lon, lat)

In [6]:
def time_reader_preload(lon, lat):
    """RasterValueReader(preload_all=True)"""
    with gps2var.RasterValueReader(DATA_PATH, preload_all=True) as reader:
        %timeit reader.get(lon, lat)

In [7]:
for scale in [0.1, 1, 10, 45]:
    print(f"scale={scale}")
    for fn in [time_rasterio_sample, time_reader, time_reader_preload]:
        print("  {:40}".format(fn.__doc__), end="")
        fn(lon * scale, lat * scale)

scale=0.1
  rasterio.DatasetReader.sample()         540 ms ± 3.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  RasterValueReader()                     8.42 ms ± 25.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
  RasterValueReader(preload_all=True)     8.04 ms ± 8.54 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
scale=1
  rasterio.DatasetReader.sample()         544 ms ± 8.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  RasterValueReader()                     9.56 ms ± 20.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
  RasterValueReader(preload_all=True)     8.61 ms ± 42.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
scale=10
  rasterio.DatasetReader.sample()         543 ms ± 4.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  RasterValueReader()                     68.4 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
  RasterValueReader(preload_all=True)     20.1 ms ± 164 µs per loop (mean ±

## Concurrent reading, interpolation

In [9]:
for kwargs in itertools.product([0.1, 10], ["nearest", "bilinear"]):
    kwargs = dict(zip(["scale", "interpolation"], kwargs))
    print(", ".join("{}={!r}".format(k, v) for k, v in kwargs.items()))
    scale = kwargs.pop("scale")
    lon_scaled, lat_scaled = lon * scale, lat * scale

    print("  sync:  ", end="")
    with gps2var.RasterValueReaderPool(DATA_PATH, num_workers=8, **kwargs) as reader:    
        %timeit [reader.get(lon_scaled, lat_scaled) for _ in range(32)]
    print("  async: ", end="")
    with gps2var.RasterValueReaderPool(DATA_PATH, num_workers=8, **kwargs) as reader:
        %timeit list(cf.as_completed(reader.async_get(lon_scaled, lat_scaled) for _ in range(32)))

scale=0.1, interpolation='nearest'
  sync:  322 ms ± 693 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
  async: 55.7 ms ± 3.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
scale=0.1, interpolation='bilinear'
  sync:  977 ms ± 28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  async: 182 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
scale=10, interpolation='nearest'
  sync:  2.45 s ± 4.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  async: 381 ms ± 19.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
scale=10, interpolation='bilinear'
  sync:  3.84 s ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  async: 575 ms ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
